In [47]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [48]:
# pickle_out=open("./pickled/Q_table_v2_15_jan","wb")
# pickle.dump(Q,pickle_out)
# pickle_out.close()
pickle_in=open("./all_jobs","rb")
all_jobs=pickle.load(pickle_in)
pickle_in.close()

In [49]:
all_jobs.job_title=all_jobs.job_title.map(lambda x: x.lower())

# note that the engineer label here refers to data engineer

data_list=['data','deep learning','machine learning',' ai','ai ','nlp ',' nlp',' ml','ml ','artificial intelligence']

def is_category(x):
    ans="dont_care"
    if any(data_word in x for data_word in set(data_list)):
        if "database" in x:
            ans="database"
        if "scientist" in x:
            ans="scientist"
        if "analyst" in x:
            ans="analyst"
        if "engineer" in x:
            ans="engineer"
    return(ans)

all_jobs["is_category"]=all_jobs.job_title.map(is_category)


In [50]:
uniquetitle, duplicate_index = [],[]
for (i,x) in enumerate(all_jobs['job_title']):

    if len(x) > 35: # if job desc is too long and repeats, likely that it is a duplicate.
        if x in set(uniquetitle):
            duplicate_index.append(i)
        elif x not in set(uniquetitle):
            uniquetitle.append(x)
all_jobs.drop(duplicate_index,inplace=True)

In [51]:
all_jobs=all_jobs[all_jobs.is_category!="dont_care"]

In [52]:
X = all_jobs.drop(["is_category"],axis=1).copy()
X.drop(["job_title"],inplace=True,axis=1)
y = all_jobs.is_category.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
# then do train test split and leave the test set untouched to avoid cheating.

# combine all descriptions for analyst
# combine all job desc for scientist

# tfidf to extract terms (or ngrams) unique to each category
# similar idea to today's trump tweets lab
# whatever numerical features engineered for train set, manually replicate on test set also.

In [54]:
countvec = CountVectorizer(lowercase=True,ngram_range=(2,4),stop_words="english")
X_trainCV = countvec.fit_transform(X_train['required_skills'])
X_testCV = countvec.transform(X_test['required_skills'])

In [55]:
X_trainCV = pd.DataFrame(X_trainCV.todense())
X_testCV = pd.DataFrame(X_testCV.todense())

In [56]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_trainCV)
X_test_ss = ss.transform(X_testCV)

In [57]:
X_pca = PCA(n_components=100)
X_trainPCA = X_pca.fit_transform(X_train_ss)
X_testPCA = X_pca.transform(X_test_ss)

In [58]:
lr = LogisticRegression()
lr.fit(X_trainPCA,y_train)
lr.score(X_testPCA,y_test)

0.6424870466321243

In [59]:
y_test.value_counts()/len(y_test)

engineer     0.362694
scientist    0.316062
analyst      0.295337
database     0.025907
Name: is_category, dtype: float64

In [60]:
X_pca.explained_variance_ratio_.sum()

0.55504341663569234

In [61]:
y_pred_lg=lr.predict(X_testPCA)
print(f"""
{metrics.classification_report(y_test, y_pred_lg)}
{metrics.confusion_matrix(y_test, y_pred_lg)}
""")


             precision    recall  f1-score   support

    analyst       0.53      0.86      0.66        57
   database       1.00      0.40      0.57         5
   engineer       0.71      0.43      0.54        70
  scientist       0.75      0.70      0.73        61

avg / total       0.68      0.64      0.63       193

[[49  0  5  3]
 [ 3  2  0  0]
 [29  0 30 11]
 [11  0  7 43]]

